
# Update recipes

Below are some ad-hoc code chunks to assist me in mass-updating conda-forge R recipes

In [1]:
import os
import sys
import time

In [2]:
# Authenticate with GitHub

import github

gh_token = os.environ['GH_TOKEN']
gh = github.Github(gh_token)
gh_me = gh.get_user()
print("Authenticated as %s"%(gh_me.login))

Authenticated as jdblischak


In [25]:
# Check rate limit
gh.get_rate_limit()

RateLimit(core=Rate(reset=2022-01-02 21:07:42, remaining=4791, limit=5000))

In [30]:
# Search
# https://pygithub.readthedocs.io/en/latest/github.html?highlight=search#github.MainClass.Github.search_code
results = gh.search_code(query="org:conda-forge jdblischak language:YAML later")
results.totalCount

11

In [27]:
# Repositories to keep
skip = ['r-feedstock',
        'r-base-feedstock',
        'r-git2r-feedstock',
        'r-knitr-feedstock',
        'r-rmarkdown-feedstock',
        'r-workflowr-feedstock', 'r-reshape-feedstock']

In [31]:
# Convert search results to dict of repository objects
dict_repo = {}
for r in results:
    repo = r.repository
    name_repo = os.path.basename(repo.full_name)
    if name_repo in skip:
        continue
    if name_repo[:2] != "r-":
        continue
    dict_repo[name_repo] = repo
print(len(dict_repo))

10


In [32]:
# Save list of packages to generate with helper script
dir_helper = "../../conda_r_skeleton_helper"
if not os.path.exists(dir_helper):
    sys.stderr.write("Need to clone helper repo")
    sys.exit(1)
file_packages = dir_helper + "/packages.txt"
handle_packages = open(file_packages, "w")
for name_temp in dict_repo.keys():
    handle_packages.write(name_temp[:-10] + "\n")
handle_packages.close()

In [33]:
%%bash -s {file_packages}

wc -l $1

10 ../../conda_r_skeleton_helper/packages.txt


In [35]:
# Fork repositories
for repo in dict_repo.values():
    print(repo.full_name)
    gh_me.create_fork(repo)
    time.sleep(0.5)

conda-forge/r-gbrd-feedstock
conda-forge/r-dynamictreecut-feedstock
conda-forge/r-readbrukerflexdata-feedstock
conda-forge/r-lokern-feedstock
conda-forge/r-mpm-feedstock
conda-forge/r-densityclust-feedstock
conda-forge/r-rsvgtipsdevice-feedstock
conda-forge/r-rtensor-feedstock
conda-forge/r-readmzxmldata-feedstock
conda-forge/r-idr-feedstock


In [45]:
%%bash -s {file_packages}

# Clone repositories

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  if [ ! -d "$localdir" ]
  then
    git clone --quiet git@github.com:jdblischak/${pkg}-feedstock.git "$localdir"
  fi
done

r-gbrd
r-dynamictreecut
r-readbrukerflexdata
r-lokern
r-mpm
r-densityclust
r-rsvgtipsdevice
r-rtensor
r-readmzxmldata
r-idr


In [37]:
%%bash -s {dir_helper}

# Generate new recipes

# Had to manually set `--allow-archived` flag to regenerate old recipes for
# archived packages

cd "$1"
Rscript run.R

Processing r-gbrd
Adding in variants from internal_defaults
Parsing input package r-gbrd:
.. name: gbrd location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-gbrd
Making/refreshing recipe for gbrd
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-gbRd_0.4-11_0251f6dd6c.tar.gz
Writing recipe for gbrd
--dirty flag and --keep-old-work not specified. Removing build/test folder after successful build/test.

Processing r-dynamictreecut
Adding in variants from internal_defaults
Parsing input package r-dynamictreecut:
.. name: dynamictreecut location: None new_location: /home/jdblischak/repos/conda_r_skeleton_helper/r-dynamictreecut
Making/refreshing recipe for dynamictreecut
Fetching main index from https://cran.r-project.org
Reading package metadata from /home/jdblischak/miniconda3/conda-bld/src_cache/source-dynamicTreeCut_1.63-1_831307f64e.tar.gz
Writing recipe for dynamictreecut
-

INFO:conda_build.variants:Adding in variants from internal_defaults
/home/jdblischak/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/jdblischak/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cran.r-project.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
No hash (md5, sha1, sha256) provided for source-gbRd_0.4-11.tar.gz.  Source download forced.  Add hash to recipe to use source cache.
INFO:conda_build.config:--dirty flag and --keep-old-work not specified. Removing build/test folder after successful build/test.

INFO:cond

In [46]:
# Get maintainers to add back

dict_maintainers = {}

conda_forge_r = {
    "conda-forge/r",
    "johanneskoester",
    "bgruening",
    "daler",
    "jdblischak",
    "cbrueffer",
    "dbast",
    "dpryan79",
    # typo from r-diffr-feedstock'
    "Johanneskoester",
    "Bgruening",
    "Daler",
    "Jdblischak",
    "Cbrueffer",
    "Dbast",
    "Dpryan79"
}

for name_temp in dict_repo.keys():
    file_codeowners = "/tmp/" + name_temp + "/.github/CODEOWNERS"
    handle_codeowners = open(file_codeowners, "r")
    codeowners = handle_codeowners.readlines()
    handle_codeowners.close()
    codeowners = codeowners[0][2:].replace("@", "").split(" ")
    maintainers = set(codeowners).difference(conda_forge_r)
    dict_maintainers[name_temp] = maintainers

In [47]:
# Increment original build numbers

dict_builds = {}

for name_temp in dict_repo.keys():
    file_meta = "/tmp/" + name_temp + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    for line in handle_meta:
        if line.strip()[:6] == "number":
            build_num = int(line.strip().split(" ")[1])
            build_num = build_num + 1
            dict_builds[name_temp] = build_num
    handle_meta.close()

In [48]:
dict_builds

{'r-gbrd-feedstock': 1003,
 'r-dynamictreecut-feedstock': 1004,
 'r-readbrukerflexdata-feedstock': 1004,
 'r-lokern-feedstock': 1,
 'r-mpm-feedstock': 1004,
 'r-densityclust-feedstock': 1005,
 'r-rsvgtipsdevice-feedstock': 1004,
 'r-rtensor-feedstock': 1,
 'r-readmzxmldata-feedstock': 1004,
 'r-idr-feedstock': 1004}

In [49]:
# Obtain original license field

dict_licenses = {}

for name_temp in dict_repo.keys():
    file_meta = "/tmp/" + name_temp + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    for line in handle_meta:
        if line.strip()[:8] == "license:":
            license_field = line.strip().split(": ")[1]
            dict_licenses[name_temp] = license_field
    handle_meta.close()
dict_licenses

{'r-gbrd-feedstock': 'GPL-2.0-or-later',
 'r-dynamictreecut-feedstock': 'GPL-2.0-or-later',
 'r-readbrukerflexdata-feedstock': 'GPL-3.0-or-later',
 'r-lokern-feedstock': 'GPL-2.0-or-later',
 'r-mpm-feedstock': 'GPL-2.0-or-later',
 'r-densityclust-feedstock': 'GPL (>= 2)',
 'r-rsvgtipsdevice-feedstock': 'GPL-2.0-or-later',
 'r-rtensor-feedstock': 'GPL-3.0-or-later',
 'r-readmzxmldata-feedstock': 'GPL-3.0-or-later',
 'r-idr-feedstock': 'GPL-2.0-or-later'}

In [50]:
%%bash -s {dir_helper}

# Copy updated recipes to cloned repositories

pkgs="$1/packages.txt"
for pkg in `cat $pkgs`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  # For some reason globbing doesn't work when run from within the notebook
  cp "$1/$pkg/meta.yaml" "$localdir/recipe/"
  cp "$1/$pkg/build.sh" "$localdir/recipe/"
  cp "$1/$pkg/bld.bat" "$localdir/recipe/"
done

r-gbrd
r-dynamictreecut
r-readbrukerflexdata
r-lokern
r-mpm
r-densityclust
r-rsvgtipsdevice
r-rtensor
r-readmzxmldata
r-idr


In [51]:
# Add back maintainers, bump version number, and fix license

for feedstock in dict_maintainers.keys():
    maintainers = dict_maintainers[feedstock]
    print(feedstock)
    file_meta = "/tmp/" + feedstock + "/recipe/meta.yaml"
    handle_meta = open(file_meta, "r")
    meta = handle_meta.readlines()
    handle_meta.close()
    maintainers_formatted = ["    - " + id + "\n" for id in maintainers]
    lines_out = meta
    for i in range(len(meta)):
        line = meta[i]
        if line.rstrip() == "    - conda-forge/r" and (len(maintainers)) != 0:
            lines_out = meta[:i+1] + maintainers_formatted + meta[i+1:]
    for i in range(len(lines_out)):
        line = lines_out[i]
        if line.rstrip() == "  number: 0":
            lines_out[i] = "  number: %d\n"%(dict_builds[feedstock])
    for i in range(len(lines_out)):
        line = lines_out[i]
        if line[:10] == "  license:":
            lines_out[i] = "  license: %s\n"%(dict_licenses[feedstock])
    handle_meta = open(file_meta, "w")
    handle_meta.write("".join(lines_out))
    handle_meta.close()

r-gbrd-feedstock
r-dynamictreecut-feedstock
r-readbrukerflexdata-feedstock
r-lokern-feedstock
r-mpm-feedstock
r-densityclust-feedstock
r-rsvgtipsdevice-feedstock
r-rtensor-feedstock
r-readmzxmldata-feedstock
r-idr-feedstock


In [52]:
%%bash -s {file_packages}

# Commit updated recipes to branch "update-recipe"

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git checkout -b "update-recipe"
  git add recipe/
  git commit -m "Update recipe"
done

r-gbrd
[update-recipe e130b74] Update recipe
 3 files changed, 32 insertions(+), 28 deletions(-)
r-dynamictreecut
[update-recipe 20922bb] Update recipe
 3 files changed, 34 insertions(+), 28 deletions(-)
 rewrite recipe/build.sh (97%)
r-readbrukerflexdata
[update-recipe e9a7e44] Update recipe
 3 files changed, 36 insertions(+), 28 deletions(-)
 rewrite recipe/build.sh (97%)
r-lokern
[update-recipe 67cbe3b] Update recipe
 3 files changed, 39 insertions(+), 25 deletions(-)
r-mpm
[update-recipe d2537b7] Update recipe
 3 files changed, 36 insertions(+), 34 deletions(-)
r-densityclust
[update-recipe 868cabb] Update recipe
 3 files changed, 43 insertions(+), 25 deletions(-)
r-rsvgtipsdevice
[update-recipe 3340e75] Update recipe
 3 files changed, 32 insertions(+), 35 deletions(-)
 rewrite recipe/build.sh (97%)
r-rtensor
[update-recipe ba40493] Update recipe
 3 files changed, 46 insertions(+), 22 deletions(-)
r-readmzxmldata
[update-recipe 4c41093] Update recipe
 3 files changed, 38 insertions

Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'
Switched to a new branch 'update-recipe'


In [53]:
%%bash -s {file_packages}

# Rerender feedstock

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  conda-smithy rerender -c auto
done

r-gbrd
Adding in variants from internal_defaults
Adding in variants from /tmp/tmp_b3dy9jj/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe b805394] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2022.01.02.17.59.50
 10 files changed, 34 insertions(+), 42 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-dynamictreecut
Adding in variants from internal_defaults
Adding in variants from /tmp/tmptseg4roz/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe a564811] MNT: Re-rendered with conda-build 3.21.7, conda-smithy 3.16.1, and conda-forge-pinning 2022.01.02.17.59.50
 10 files changed, 35 insertions(+), 43 deletions(-)
 delete mode 100644 .ci_support/migrations/r410.yaml
r-readbrukerflexdata
Adding in variants from internal_defaults
Adding in variants from /tmp/tmpy554obzz/conda_build_config.yaml
Adding in variants from argument_variants
[update-recipe 69887cd] MNT: Re-re

INFO:conda_smithy.configure_feedstock:Downloading conda-forge-pinning-2022.01.02.17.59.50
INFO:conda_smithy.configure_feedstock:Extracting conda-forge-pinning to /tmp/tmp_b3dy9jj
INFO:conda_smithy.configure_feedstock:r410.yaml is closed now. Removing
Setting build platform. This is only useful when pretending to be on another platform, such as for rendering necessary dependencies on a non-native platform. I trust that you know what you're doing.
Setting build arch. This is only useful when pretending to be on another arch, such as for rendering necessary dependencies on a non-native arch. I trust that you know what you're doing.
No numpy version specified in conda_build_config.yaml.  Falling back to default numpy value of 1.16
INFO:conda_build.variants:Adding in variants from internal_defaults
INFO:conda_build.variants:Adding in variants from /tmp/tmp_b3dy9jj/conda_build_config.yaml
INFO:conda_build.variants:Adding in variants from argument_variants
/home/jdblischak/miniconda3/lib/pyth

In [54]:
%%bash -s {file_packages}

# Push to GitHub

for pkg in `cat $1`
do
  echo $pkg
  localdir="/tmp/${pkg}-feedstock"
  cd "$localdir"
  git push origin update-recipe
done

r-gbrd
r-dynamictreecut
r-readbrukerflexdata
r-lokern
r-mpm
r-densityclust
r-rsvgtipsdevice
r-rtensor
r-readmzxmldata
r-idr


remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-gbrd-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-gbrd-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-dynamictreecut-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-dynamictreecut-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-readbrukerflexdata-feedstock/pull/new/update-recipe        
remote: 
To github.com:jdblischak/r-readbrukerflexdata-feedstock.git
 * [new branch]      update-recipe -> update-recipe
remote: 
remote: Create a pull request for 'update-recipe' on GitHub by visiting:       

In [55]:
# Open pull requests
# https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html?highlight=create_pull#github.Repository.Repository.create_pull
# https://docs.github.com/en/rest/reference/pulls#create-a-pull-request
# https://pygithub.readthedocs.io/en/latest/examples/PullRequest.html?highlight=create%20a%20pull%20request#create-a-new-pull-request
feedstocks = list(dict_repo.keys())
# feedstocks_completed = ["conda-forge/" + stock for stock in feedstocks]
for repo in dict_repo.values():
    # if repo.full_name in feedstocks_completed:
    #     continue
    print(repo.full_name)
    body = """Checklist
* [x] Used a [personal fork of the feedstock to propose changes](https://conda-forge.org/docs/maintainer/updating_pkgs.html#forking-and-pull-requests)
* [x] Bumped the build number (if the version is unchanged)
* [ ] Reset the build number to `0` (if the version changed)
* [x] [Re-rendered]( https://conda-forge.org/docs/maintainer/updating_pkgs.html#rerendering-feedstocks ) with the latest `conda-smithy` (Use the phrase <code>@<space/>conda-forge-admin, please rerender</code> in a comment in this PR for automated rerendering)
* [x] Ensured the license file is being packaged.

* I already rerendered locally
"""
    repo.create_pull(title="Update recipe", body=body, base="master", head="jdblischak:update-recipe")
    time.sleep(1)

conda-forge/r-gbrd-feedstock
conda-forge/r-dynamictreecut-feedstock
conda-forge/r-readbrukerflexdata-feedstock
conda-forge/r-lokern-feedstock
conda-forge/r-mpm-feedstock
conda-forge/r-densityclust-feedstock
conda-forge/r-rsvgtipsdevice-feedstock
conda-forge/r-rtensor-feedstock
conda-forge/r-readmzxmldata-feedstock
conda-forge/r-idr-feedstock
